In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
pip install --upgrade tables

In [ ]:
import pickle
import pandas as pd
import numpy as np
import cv2
import os

In [ ]:
contour_dirs = ['/content/drive/MyDrive/BendeskyLab/2FishEncounterDL/IM1_IM2_2.1.1_L/curve_scores',
                '/content/drive/MyDrive/BendeskyLab/2FishEncounterDL/IM1_IM2_2.1.1_R/curve_scores',
                '/content/drive/MyDrive/BendeskyLab/2FishEncounterDL/VM3_VM4_5.1.1_L/curve_scores',
                '/content/drive/MyDrive/BendeskyLab/2FishEncounterDL/IM2_IM4_5.1.2_R/curve_scores',
                '/content/drive/MyDrive/BendeskyLab/2FishEncounterDL/VM1_VM3_1.1.1_L/curve_scores',
                '/content/drive/MyDrive/BendeskyLab/2FishEncounterDL/VM1_VM3_1.1.1_R/curve_scores',
                '/content/drive/MyDrive/BendeskyLab/2FishEncounterDL/IM2_IM4_5.1.2_L/curve_scores',
                '/content/drive/MyDrive/BendeskyLab/2FishEncounterDL/VM3_VM4_5.1.1_R/curve_scores']
head_index_dirs = ['/content/drive/MyDrive/BendeskyLab/2FishEncounterDL/IM1_IM2_2.1.1_L/head_index',
                   '/content/drive/MyDrive/BendeskyLab/2FishEncounterDL/IM1_IM2_2.1.1_R/head_index',
                   '/content/drive/MyDrive/BendeskyLab/2FishEncounterDL/VM3_VM4_5.1.1_L/head_index',
                   '/content/drive/MyDrive/BendeskyLab/2FishEncounterDL/IM2_IM4_5.1.2_R/head_index',
                   '/content/drive/MyDrive/BendeskyLab/2FishEncounterDL/VM1_VM3_1.1.1_L/head_index',
                   '/content/drive/MyDrive/BendeskyLab/2FishEncounterDL/VM1_VM3_1.1.1_R/head_index',
                   '/content/drive/MyDrive/BendeskyLab/2FishEncounterDL/IM2_IM4_5.1.2_L/head_index',
                   '/content/drive/MyDrive/BendeskyLab/2FishEncounterDL/VM3_VM4_5.1.1_R/head_index']

In [ ]:
def find_centroid(contour):
    moments=cv2.moments(contour)
    m01=moments.get("m01")
    m10=moments.get("m10")
    m00=moments.get("m00")
    xbar=m10/m00
    ybar=m01/m00
    return xbar,ybar

In [ ]:
START_FRAME = 72000
END_FRAME = 144000

In [ ]:
base_df = pd.read_csv(base_features[1])
path = '/content/drive/MyDrive/DSI_h5/Training_Test_Set/training/h5/VM1_VM31.1.1_LDLC_resnet50_DLC_toptrackFeb27shuffle1_170000.h5'
f = pd.HDFStore(path,'r')
df = f.get('df_with_missing')
df.columns = df.columns.droplevel()
head_x_pos,head_y_pos = df['A_head']['x'],df['A_head']['y']

In [ ]:
df

bodyparts      A_head              ... Q_analtipback           
coords              x           y  ...             y likelihood
0          204.754974  101.762920  ...    126.299040   0.075136
1          212.404705  100.697269  ...    124.149322   0.009433
2          224.474854  103.753138  ...    126.096437   0.033278
3          282.574026   70.465736  ...    124.761740   0.063795
4          278.421072   67.738428  ...    131.812851   0.010598
...               ...         ...  ...           ...        ...
216057     454.164353   86.580655  ...    171.984650   0.227157
216058     455.594519   84.812183  ...    166.204627   0.019648
216059     452.876843   84.986449  ...    170.461310   0.175631
216060     454.077887   83.257028  ...    170.694764   0.021984
216061     452.995513   83.409160  ...    168.919092   0.029824

[216062 rows x 51 columns]

In [ ]:
def concat_features(base_features, DLC_path,contours_path,tail_index_path,tail_angle_path,tail_dev_path):
  fish_name = os.path.basename(os.path.dirname(base_features))
  print(fish_name)
  base_df = pd.read_csv(base_features)
  path = DLC_path
  f = pd.HDFStore(path,'r')
  df = f.get('df_with_missing')
  df.columns = df.columns.droplevel()
  head_x_pos,head_y_pos = df['A_head']['x'],df['A_head']['y']
  head_x_pos = np.array(head_x_pos.iloc[START_FRAME:END_FRAME])
  head_y_pos = np.array(head_y_pos.iloc[START_FRAME:END_FRAME])
  base_df['head_x'] = head_x_pos
  base_df['head_y'] = head_y_pos
  with open(contours_path,"rb") as fp:
    contours = pickle.load(fp)
  with open(tail_index_path,"rb") as fp:
    tail_indexs = pickle.load(fp)
  centroid_x = []
  centroid_y = []
  tail_x = []
  tail_y = []
  for tail_index,contour in zip(tail_indexs,contours):
    t_x,t_y = contour[tail_index%len(contour),:2]
    contour = np.array(contour[:,:2][:,None,:],dtype = np.int)
    xbar,ybar = find_centroid(contour)
    centroid_x.append(xbar)
    centroid_y.append(ybar)
    tail_x.append(t_x)
    tail_y.append(t_y)
  base_df['centroid_x'] = centroid_x
  base_df['centroid_y'] = centroid_y
  base_df['tail_x'] = tail_x
  base_df['tail_y'] = tail_y
  with open(tail_angle_path,"rb") as fp:
    tail_angle = pickle.load(fp)
  base_df['tail_angle'] = tail_angle
  with open(tail_dev_path,"rb") as fp:
    tail_dev = pickle.load(fp)
  base_df['tail_dev'] = tail_dev
  if 'L' in fish_name:
      print("flipping coordinate for {}".format(fish_name))
      base_df['orientation'] = 180-base_df['orientation']
      base_df['head_x'] = 500-base_df['head_x']
      base_df['centroid_x'] = 500-base_df['centroid_x']
      base_df['tail_x'] = 500-base_df['tail_x']
  return base_df

In [ ]:
base_features = ['/content/drive/MyDrive/BendeskyLab/2FishEncounterDL/IM1_IM2_2.1.1_L/base_features.csv',
                 '/content/drive/MyDrive/BendeskyLab/2FishEncounterDL/IM1_IM2_2.1.1_R/base_features.csv']
DLC_paths = ['/content/drive/MyDrive/DSI_h5/Training_Test_Set/training/h5/IM1_IM2_2.1.1_LDLC_resnet50_DLC_toptrackFeb27shuffle1_170000.h5',
             '/content/drive/MyDrive/DSI_h5/Training_Test_Set/training/h5/IM1_IM2_2.1.1_RDLC_resnet50_DLC_toptrackFeb27shuffle1_170000.h5']
contours_paths = ['/content/drive/MyDrive/BendeskyLab/2FishEncounterDL/IM1_IM2_2.1.1_L/curve_scores',
                  '/content/drive/MyDrive/BendeskyLab/2FishEncounterDL/IM1_IM2_2.1.1_R/curve_scores']
tail_indexs = ['/content/drive/MyDrive/BendeskyLab/2FishEncounterDL/IM1_IM2_2.1.1_L/tail_index',
               '/content/drive/MyDrive/BendeskyLab/2FishEncounterDL/IM1_IM2_2.1.1_R/tail_index']
tail_angle_paths = ['/content/drive/MyDrive/BendeskyLab/2FishEncounterDL/IM1_IM2_2.1.1_L/tailAngle',
                    '/content/drive/MyDrive/BendeskyLab/2FishEncounterDL/IM1_IM2_2.1.1_R/tailAngle']
tail_dev_paths = ['/content/drive/MyDrive/BendeskyLab/2FishEncounterDL/IM1_IM2_2.1.1_L/tail_dev',
                  '/content/drive/MyDrive/BendeskyLab/2FishEncounterDL/IM1_IM2_2.1.1_R/tail_dev']



In [ ]:
len(base_features),len(DLC_paths),len(contours_paths),len(tail_indexs),len(tail_angle_paths),len(tail_dev_paths)

(6, 6, 6, 6, 6, 6)

In [ ]:
new_df = concat_features(base_features[1],DLC_paths[1],contours_paths[1],tail_indexs[1],tail_angle_paths[1],tail_dev_paths[1])

In [ ]:
import os
new_df.to_csv(os.path.join(os.path.dirname(base_features[1]),'IM1_IM2_2.1.1_R_data_auto_scored.csv'))

**Rembemer to flip the left fish afterwards, 180-orientation, 500-head_x, 500 - centroid_x, 500-tail_x**

In [ ]:
from tqdm import tqdm
for i in tqdm(range(len(base_features))):
  new_df = concat_features(base_features[i],DLC_paths[i],contours_paths[i],tail_indexs[i],tail_angle_paths[i],tail_dev_paths[i])
  new_df.to_csv(os.path.join(os.path.dirname(base_features[i]), os.path.basename(os.path.dirname(base_features[i]))+'_data_auto_scored.csv'))

  0%|          | 0/6 [00:00<?, ?it/s]

1.2.2L
flipping coordinate for 1.2.2L


 17%|█▋        | 1/6 [00:04<00:22,  4.41s/it]

1.2.2R


 33%|███▎      | 2/6 [00:08<00:17,  4.34s/it]

...

 67%|██████▋   | 4/6 [00:16<00:08,  4.04s/it]

1.4.1L
flipping coordinate for 1.4.1L


 83%|████████▎ | 5/6 [00:20<00:04,  4.02s/it]

1.4.1R


100%|██████████| 6/6 [00:24<00:00,  4.05s/it]


In [ ]:
new_df

,operculum,orientation,movement_speed,turning_angle,head_x,head_y,centroid_x,centroid_y,tail_x,tail_y,tail_angle,tail_dev
0,68.333862,163.717090,59.729040,37.586639,55.670138,420.823826,118.586096,409.053396,210.0,405.0,172.629830,13.095892
1,65.666786,167.710300,59.729040,37.586639,55.863207,419.394465,118.599538,409.057482,210.0,405.0,172.629830,13.095892
2,68.030919,160.582429,59.729040,37.586639,54.789345,422.339467,118.545217,409.041152,210.0,405.0,172.629830,13.095892
3,68.030919,160.582429,59.729040,37.586639,55.996053,419.314075,118.599538,409.057482,210.0,405.0,172.629830,13.095892
4,68.030919,160.582429,59.729040,37.586639,55.463128,420.622584,118.586096,409.053396,210.0,405.0,172.629830,13.095892
...,...,...,...,...,...,...,...,...,...,...,...,...
71995,121.313348,171.995956,154.453672,60.114252,54.943475,131.566303,97.677280,163.006241,75.0,216.0,46.145763,74.872575
71996,121.313348,171.995956,154.453672,74.016793,52.046900,134.485026,99.690064,163.391086,77.0,214.0,45.000000,70.605605
71997,121.313348,171.995956,154.453672,71.809166,59.796105,121.755528,101.358006,163.386285,80.0,213.0,41.715175,68.280832
71998,126.456696,149.128772,45.255372,72.259288,53.443760,135.573509,102.704499,163.026631,85.0,212.0,45.092412,65.936822
